In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain.chat_models import ChatOpenAI


In [4]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [5]:
KEY=os.getenv("OPENAI_API_KEY")

In [6]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

d:\Python\Lecture6\IntelliMCQ\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [9]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"], #varaibles user will pass
    template=TEMPLATE
    )
#Input prompt

In [11]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

#creating a chain: Connection between LLM and prompt

In [12]:
#Prompt for quiz evaluaion

TEMPLATE2="""
As an expert in English grammar and writing, your task is to analyze a multiple-choice quiz designed for {subject} students.\
Provide a concise evaluation of the quiz's complexity within 50 words, assigning it a score based on cognitive and analytical levels.
MCQ Quiz:{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)
# quiz_evaluation_prompt=PromptTemplate(input_variables=["subject"], template=TEMPLATE)

In [14]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [15]:
#Now connect both chain using sequential chain

In [16]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [17]:
#now we need a data on the top of which we will create a quiz
#make file data.txt and read the data in file

In [18]:
file_path = os.getenv("FILE_PATH")

In [19]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [20]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [21]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [22]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"


In [23]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

d:\Python\Lecture6\IntelliMCQ\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [24]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1429
Prompt Tokens:1024
Completion Tokens:405
Total Cost:0.002346


In [25]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [26]:
quiz=response.get("quiz")

In [27]:
quiz

'\n{\n    "1": {\n        "mcq": "What is the scientific study of life called?",\n        "options": {\n            "a": "Chemistry",\n            "b": "Biology",\n            "c": "Physics",\n            "d": "Geology"\n        },\n        "correct": "b"\n    },\n    "2": {\n        "mcq": "Which theme in biology explains the unity and diversity of life?",\n        "options": {\n            "a": "Evolution",\n            "b": "Photosynthesis",\n            "c": "Mitosis",\n            "d": "Respiration"\n        },\n        "correct": "a"\n    },\n    "3": {\n        "mcq": "What allows organisms to move, grow, and reproduce?",\n        "options": {\n            "a": "Energy processing",\n            "b": "Cell division",\n            "c": "Genetic mutation",\n            "d": "Photosynthesis"\n        },\n        "correct": "a"\n    },\n    "4": {\n        "mcq": "What is the basic unit of life that processes hereditary information?",\n        "options": {\n            "a": "Atoms",\

In [28]:
review=response.get("review")

In [29]:
review

'The multiple-choice quiz designed for biology students is relatively simple in complexity. It primarily tests basic knowledge and understanding of core biological concepts. The questions mostly require recall and recognition rather than higher-order thinking skills. Score: 3/10.'

In [ ]:
quiz=json.loads(quiz)

In [ ]:
quiz

{'1': {'mcq': 'Which theme in biology explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Energy processing',
   'c': 'Cell structure',
   'd': 'Internal regulation'},
  'correct': 'a'},
 '2': {'mcq': 'What is the main function of genes in organisms?',
  'options': {'a': 'Regulate internal environments',
   'b': 'Process energy',
   'c': 'Encode hereditary information',
   'd': 'Facilitate evolution'},
  'correct': 'c'},
 '3': {'mcq': 'What is the role of biologists in studying life?',
  'options': {'a': 'Make observations and form conclusions',
   'b': 'Generate hypotheses and perform experiments',
   'c': 'Classify organisms and study ecosystems',
   'd': 'All of the above'},
  'correct': 'd'},
 '4': {'mcq': 'Which level of organization in biology involves the anatomy and physiology of plants and animals?',
  'options': {'a': 'Molecular biology',
   'b': 'Cell biology',
   'c': 'Organismal biology',
   'd': 'Population biology'},
  'correct': 'c'},
 '

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

[{'MCQ': 'Which theme in biology explains the unity and diversity of life?',
  'Choices': 'a: Evolution | b: Energy processing | c: Cell structure | d: Internal regulation',
  'Correct': 'a'},
 {'MCQ': 'What is the main function of genes in organisms?',
  'Choices': 'a: Regulate internal environments | b: Process energy | c: Encode hereditary information | d: Facilitate evolution',
  'Correct': 'c'},
 {'MCQ': 'What is the role of biologists in studying life?',
  'Choices': 'a: Make observations and form conclusions | b: Generate hypotheses and perform experiments | c: Classify organisms and study ecosystems | d: All of the above',
  'Correct': 'd'},
 {'MCQ': 'Which level of organization in biology involves the anatomy and physiology of plants and animals?',
  'Choices': 'a: Molecular biology | b: Cell biology | c: Organismal biology | d: Population biology',
  'Correct': 'c'},
 {'MCQ': 'What is the estimated age of life on Earth according to the text?',
  'Choices': 'a: 1 billion years

In [ ]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
quiz

,MCQ,Choices,Correct
0,Which theme in biology explains the unity and ...,a: Evolution | b: Energy processing | c: Cell ...,a
1,What is the main function of genes in organisms?,a: Regulate internal environments | b: Process...,c
2,What is the role of biologists in studying life?,a: Make observations and form conclusions | b:...,d
3,Which level of organization in biology involve...,a: Molecular biology | b: Cell biology | c: Or...,c
4,What is the estimated age of life on Earth acc...,a: 1 billion years | b: 2.5 billion years | c:...,c


In [ ]:
quiz.to_csv("quiz.csv",index=False)
#creates a new .csv file with the same name

In [ ]:
import logging
import os
from datetime import datetime


LOG_FILE=f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [ ]:
LOG_FILE

'04_27_2024_10_14_27.log'

In [ ]:
import os
import json
import traceback
import pandas as pd
from dotenv import load_dotenv


#import necessary packages from langchain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain #for connecting prompt template and llm
from langchain.chains import SequentialChain

